In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import numexpr as ne
import time
import copy
import sys

from tqdm import tqdm
from sklearn.metrics import accuracy_score, log_loss

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../pytorch_utils/')
from utils import evaluate, get_data, top5_accuracy, per_class_accuracy, count_params

In [ ]:
import torch.cuda
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.benchmark = True

# Load data

In [ ]:
%%time
X_train, X_val, Y_train, Y_val = get_data()
train_size = len(X_train)
val_size = len(X_val)
print(train_size, val_size)

# Load logits

In [ ]:
train_logits = np.load('/home/ubuntu/data/train_logits.npy')[()]

In [ ]:
temperature = 20.0

In [ ]:
n_models = len(train_logits)
n_models

In [ ]:
soft_targets = torch.zeros(train_size, 256)
for k in train_logits:
    soft_targets += F.softmax(torch.FloatTensor(train_logits[k])/temperature)
soft_targets /= n_models

# Create iterators

In [ ]:
from torch.utils.data.dataset import Dataset

In [ ]:
class TripleDataset(Dataset):

    def __init__(self, data_tensor, target_tensor, third_tensor):
        assert data_tensor.size(0) == target_tensor.size(0)
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
        self.third_tensor = third_tensor

    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index], self.third_tensor[index]

    def __len__(self):
        return self.data_tensor.size(0)

In [ ]:
val_data = TensorDataset(
    torch.FloatTensor(X_val), 
    torch.LongTensor(Y_val)
)

val_iterator = DataLoader(
    val_data, batch_size=64, shuffle=True, pin_memory=True
)

In [ ]:
batch_size = 32

In [ ]:
train_data = TripleDataset(
    torch.FloatTensor(X_train), 
    torch.LongTensor(Y_train),
    soft_targets
)

train_iterator = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, pin_memory=True
)

# Model

In [ ]:
sys.path.append('../squeezenet/')
from model_squeezenet import make_model

In [ ]:
model, criterion, optimizer = make_model()

In [ ]:
count_params(model)

# Train

In [ ]:
class soft_targets_logloss(nn.Module):

    def __init__(self):
        super(soft_targets_logloss, self).__init__()
        
    def forward(self, logits, targets):
        x = F.log_softmax(logits)
        return -(targets*x).sum(1).mean(0)

In [ ]:
soft_criterion = soft_targets_logloss()
logloss_weight = 0.1

In [ ]:
def train(model, criterion, optimizer, x_batch, y_batch, soft_y_batch):

    x_batch = Variable(x_batch.cuda())
    y_batch = Variable(y_batch.cuda(async=True))
    soft_y_batch = Variable(soft_y_batch.cuda(async=True))
    
    logits = model(x_batch)

    # compute logloss
    logloss = criterion(logits, y_batch)
    batch_loss = logloss.data[0]

    # compute accuracy
    pred = F.softmax(logits).max(1)[1]
    batch_accuracy = pred.eq(y_batch).float().mean().data[0]

    # compute loss
    loss = logloss_weight*logloss + soft_criterion(logits/temperature, soft_y_batch)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return batch_loss, batch_accuracy

In [ ]:
n_epochs = 6
validation_step = 200
n_batches = int(np.ceil(train_size/batch_size))
M = 3
T = n_batches*n_epochs
initial = 0.01
n_batches

In [ ]:
def lr_scheduler(optimizer, step):
    
    global initial
    decay = np.cos(np.pi*((step - 1) % (T // M))/(T // M)) + 1.0
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = initial*decay/2.0
    
    if (step - 1) % (T // M) == 0 and step != 1:
        initial *= 0.9
        print('lr is reset:', initial)
        
    return optimizer

In [ ]:
all_losses = []
all_models = []

In [ ]:
%%time
running_loss = 0.0
running_accuracy = 0.0 
start = time.time()
model.train()

for epoch in range(0, n_epochs):
    for step, (x_batch, y_batch, soft_y_batch) in enumerate(train_iterator, 1 + epoch*n_batches):
        
        optimizer = lr_scheduler(optimizer, step)
        batch_loss, batch_accuracy = train(
            model, criterion, optimizer, 
            x_batch, y_batch, soft_y_batch
        )
        running_loss += batch_loss
        running_accuracy += batch_accuracy
        
        if step % validation_step == 0:
            model.eval()
            test_loss, test_accuracy = evaluate(
                model, criterion, val_iterator
            )
            end = time.time()
            
            print('{0:.2f}  {1:.3f} {2:.3f}  {3:.3f} {4:.3f}  {5:.3f}'.format(
                step/n_batches, running_loss/validation_step, test_loss, 
                running_accuracy/validation_step, test_accuracy, end - start
            ))
            all_losses += [(
                step/n_batches,
                running_loss/validation_step, test_loss, 
                running_accuracy/validation_step, test_accuracy
            )] 
            
            running_loss = 0.0
            running_accuracy = 0.0 
            start = time.time()
            model.train()
            
        if step % (T // M) == 0:
            
            print('saving')
            model.cpu()
            clone = copy.deepcopy(model)
            all_models += [clone.state_dict()]
            model.cuda()

# Loss/epoch

In [ ]:
epochs = [x[0] for x in all_losses]
plt.plot(epochs, [x[1] for x in all_losses], label='train');
plt.plot(epochs, [x[2] for x in all_losses], label='test');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('loss');

In [ ]:
plt.plot(epochs, [x[3] for x in all_losses], label='train');
plt.plot(epochs, [x[4] for x in all_losses], label='test');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('accuracy');

# Predict val. set

In [ ]:
val_predictions = []

In [ ]:
val_iterator_no_shuffle = DataLoader(
    val_data, batch_size=32, shuffle=False, pin_memory=True
)

In [ ]:
%%time
# model.load_state_dict(all_models[-1])
model.eval()

for x_batch, _ in val_iterator_no_shuffle:

    x_batch = Variable(x_batch.cuda(), volatile=True)
    logits = model(x_batch)

    # compute probabilities
    probs = F.softmax(logits) 
    val_predictions += [probs.cpu().data.numpy()]

In [ ]:
val_predictions = np.concatenate(val_predictions, axis=0)

In [ ]:
accuracy_score(Y_val, val_predictions.argmax(1))

In [ ]:
log_loss(Y_val, val_predictions)

# Try ensemble

In [ ]:
ensemble_predictions = {}

In [ ]:
%%time
for i, state in enumerate(all_models):
    
    model.load_state_dict(state)
    ensemble_predictions[i] = []
    model.eval()
    
    for x_batch, _ in val_iterator_no_shuffle:

        x_batch = Variable(x_batch.cuda(), volatile=True)
        logits = model(x_batch)

        # compute probabilities
        probs = F.softmax(logits) 
        ensemble_predictions[i] += [probs.cpu().data.numpy()]

In [ ]:
ensemble_predictions = {
    p: np.concatenate(ensemble_predictions[p], axis=0) 
    for p in ensemble_predictions
}

In [ ]:
predictions = np.stack(ensemble_predictions.values()).mean(0)

In [ ]:
accuracy_score(Y_val, predictions.argmax(1))

In [ ]:
log_loss(Y_val, predictions)

In [ ]:
top5_accuracy(Y_val, predictions)

In [ ]:
per_class_acc = per_class_accuracy(Y_val, predictions)
per_class_acc

# Error analysis

In [ ]:
erroneous = Y_val != predictions.argmax(1)
n_errors = len(Y_val[erroneous])
n_errors

In [ ]:
to_show = np.random.choice(np.arange(0, n_errors), size=30, replace=False)

In [ ]:
pictures = X_val[erroneous][to_show].copy()
pictures_predictions = predictions.argmax(1)[erroneous][to_show]
pictures_probs = predictions.max(1)[erroneous][to_show]
pictures_true = Y_val[erroneous][to_show]

In [ ]:
mean = np.array([0.485, 0.456, 0.406], dtype='float32')
std = np.array([0.229, 0.224, 0.225], dtype='float32')
decode = np.load('../utils/decode.npy')[()]

In [ ]:
pictures = np.transpose(pictures, axes=(0, 2, 3, 1))
ne.evaluate('pictures*std', out=pictures);
ne.evaluate('pictures + mean', out=pictures);

In [ ]:
# show pictures, predicted classes and probabilities
_, axes = plt.subplots(nrows=6, ncols=5, figsize=(14, 16))
axes = axes.flatten()
for i, pic in enumerate(pictures):
    axes[i].set_axis_off();
    axes[i].imshow(pic);
    title = decode[pictures_predictions[i] + 1] + ' ' +\
        str(pictures_probs[i]) + '\ntrue: ' + decode[pictures_true[i] + 1]
    axes[i].set_title(title);
plt.tight_layout()

# Save the model

In [ ]:
for i, m in enumerate(all_models):
    torch.save(m, 'model_state' + str(i) + '.pytorch')